In [2]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer

# train datasets

df = pd.DataFrame(load_dataset("shaipro/uz_large"))



In [3]:
df

,train
0,{'conversations': '<bos><start_of_turn>user So...
1,{'conversations': '<bos><start_of_turn>user Uc...
2,{'conversations': '<bos><start_of_turn>user At...
3,{'conversations': '<bos><start_of_turn>user Ha...
4,{'conversations': '<bos><start_of_turn>user O'...
...,...
236588,{'conversations': '<bos><start_of_turn>user Si...
236589,{'conversations': '<bos><start_of_turn>user Si...
236590,{'conversations': '<bos><start_of_turn>user Si...
236591,{'conversations': '<bos><start_of_turn>user Si...
